### Projekt IUM (zestaw 09, wariant 02)

Poniższy notatnik składa się z sekcji:
1. Analiza biznesowa, kryteria sukcesu
2. Załadowanie danych do df
3. Wstępny preprocessing danych
4. Analiza danych

#### 1. Załadowanie danych do df

In [ ]:
import pandas as pd
import json

In [ ]:
def load_lines(filename):
    with open(filename) as f:
        lines = f.read().splitlines()
    return lines

In [ ]:
artists_lines = load_lines("artists.jsonl")
tracks_lines = load_lines("tracks.jsonl")